# Chemical Rocket Calculator

Dr. Daniel Duke<br>
Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)<br>
Department of Mechanical & Aerospace Engineering<br>
Monash University, Australia

In [1]:
import numpy as np
import scipy.optimize as sopt

# set physical constants
Ru = 8.314
g = 9.81

In [80]:
def solve_thrust(x):
    Mwt_exhaust, gamma_exhaust, chamber_temp, nozzle_P, chamber_P, throat_area = x
    
    if (gamma_exhaust<1.1) or (gamma_exhaust>1.4): 
        return -1e12,0,0,0,0,0,0,0,0
    
    # Exhaust gas constant
    R = Ru / Mwt_exhaust
    
    # Solve exhaust velocity
    a=((2*gamma_exhaust)/(gamma_exhaust-1))*R*1e3*chamber_temp
    b=1-(nozzle_P/chamber_P)**((gamma_exhaust-1)/gamma_exhaust)
    Cj = np.sqrt(a*b)

    # Specific impulse
    Isp = Cj/g
    
    # Isentropically expanded exhaust temperature
    Te=chamber_temp*(nozzle_P/chamber_P)**((gamma_exhaust-1)/gamma_exhaust)
    
    # Sound speed in exhaust
    ae = np.sqrt(np.abs(gamma_exhaust * R*1e3 * Te))
    
    # Mach number
    Me = Cj/ae
    
    # Density
    rho_e = nozzle_P / (R*Te)
    
    # Throat to exit area ratio
    AR=(1./Me)*((2+(gamma_exhaust-1)*Me**2)/(gamma_exhaust+1))**((gamma_exhaust+1)/(2*gamma_exhaust-2))
    
    # Exit area
    Ae = throat_area * AR
    
    # Mass flow rate
    mdot = Cj * rho_e * Ae
    
    # Thrust [kN]
    F = mdot * Cj * 1e-3
    
    return Cj, Isp, Te, Me, rho_e, AR, Ae, mdot, F

def solve_burnout(x, n_engines, burn_time, structure_mass, payload_mass, launch_angle):
    Cj, Isp, Te, Me, rho_e, AR, Ae, mdot, F = x
    
    propellant_mass = mdot * burn_time * n_engines
    m = propellant_mass + structure_mass + payload_mass # total mass
    mass_ratio = m / (structure_mass + payload_mass)
    
    a = Cj*burn_time*((np.log(1./mass_ratio)/(mass_ratio-1))+ 1)/1000.
    b = g*np.sin(launch_angle*np.pi/180.)*(burn_time**2)/2000.
    
    zb = a - b
    
    deltaV = g*Isp*np.log(mass_ratio)-g*burn_time*np.sin(launch_angle)
    
    return propellant_mass, m, mass_ratio, zb, deltaV

def print_thrust(x,y):
    Mwt_exhaust, gamma_exhaust, chamber_temp, nozzle_P, chamber_P, throat_area = x
    Cj, Isp, Te, Me, rho_e, AR, Ae, mdot, F = y
    print("----------------------------------------------------------------------")
    print("\nExhaust molar weight = %.3f kg/kmol" % Mwt_exhaust)
    print("Exhaust specific heat ratio = %.3f" % gamma_exhaust)
    print("Exhaust gas constant = %.3f" % (Ru/Mwt_exhaust))
    print("Chamber temperature = %.3f K" % chamber_temp)
    print("Nozzle exit pressure = %.3f kPa" % nozzle_P)
    print("Chamber pressure = %.3f MPa" % (chamber_P*1e-3))
    print("Throat area = %.3f m2" % throat_area)
    print("Throat diameter = %.3f m" % (np.sqrt(throat_area*4/np.pi)))    
    print("")
    print("Nozzle velocity = %.3f m/s" % Cj)
    print("Exhaust temperature = %.3f K" % Te)
    print("Mach number = %.3f" % Me)
    print("Exhaust density = %.3f kg/m3" % rho_e)
    print("Throat-Exit area ratio = %.3f" % AR)
    print("Mass flow rate propellant = %.3f kg/s" % mdot)
    print("")
    print("Specific impulse = %.3f s" % Isp)
    print("Engine Thrust = %.3f kN" % F)
    print("Exit area = %.3f m2" % Ae)
    print("Exit diameter = %.3f m" % (np.sqrt(Ae*4/np.pi)))

    return

def print_burnout(z):
    propellant_mass, m, mass_ratio, zb, deltaV = z
    print("")
    print("Propellant mass = %.3f kg" % propellant_mass)
    print("Total mass = %.3f kg" % m)
    print("Mass ratio = %.3f" % mass_ratio)
    print("Burnout height = %.3f km" % zb)
    print("Velocity increment = %.3f km/s" % (deltaV*1e-3))
    return
    
    

In [14]:
# Iterative solver for desired rocket thrust

# set initial guesses
chamber_temp = 2273
chamber_P = 15000
gamma_exhaust = 1.15
throat_area = 0.009
Mwt_exhaust = 22
init_guess = (gamma_exhaust, chamber_temp, chamber_P, throat_area, Mwt_exhaust)

# set constants
nozzle_P = 101.325
constants = {'args':(nozzle_P)}

def fwrapper(variables, *args):
    nozzle_P = args
    gamma_exhaust, chamber_temp, chamber_P, throat_area, Mwt_exhaust = variables
    x = (Mwt_exhaust, gamma_exhaust, chamber_temp, nozzle_P, chamber_P, throat_area)
    Cj, Isp, Te, Me, rho_e, AR, Ae, mdot, F = solve_thrust(x)
    
    # Return differences between found value and target values
    res = [ 10*((F-25)/25.)**2, ((Ae-0.049087)/0.049087)**2, 100*((Isp - 311)/311.)**2 ]
    return np.sum(res)

# set bounds on variables
class MyBounds(object):
     #gamma_exhaust, chamber_temp, chamber_P, throat area, mol wt
     def __init__(self, xmax=(1.333,3623,30000,0.049,26), xmin=(1.05,1000,1000,0.0005,14) ):
         self.xmax = np.array(xmax)
         self.xmin = np.array(xmin)
     def __call__(self, **kwargs):
         x = kwargs["x_new"]
         tmax = bool(np.all(x <= self.xmax))
         tmin = bool(np.all(x >= self.xmin))
         return tmax and tmin

mybounds = MyBounds()
# Run basin-hopping
result = sopt.basinhopping(fwrapper, init_guess, minimizer_kwargs=constants,accept_test=mybounds,\
                           niter=1000,interval=10,stepsize=0.05,disp=False)
gamma_exhaust, chamber_temp, chamber_P, throat_area, Mwt_exhaust = result.x

# Update and re-run with final values.
x = (Mwt_exhaust, gamma_exhaust, chamber_temp, nozzle_P, chamber_P, throat_area)
y = solve_thrust(x)
print_thrust(x,y)


----------------------------------------------------------------------

Exhaust molar weight = 16.274 kg/kmol
Exhaust specific heat ratio = 1.100
Chamber temperature = 2273.915 K
Nozzle exit pressure = 101.325 kPa
Chamber pressure = 14.852 MPa
Throat area = 0.001 m2
Throat diameter = 0.035 m

Nozzle velocity = 3052.310 m/s
Exhaust temperature = 1444.845 K
Mach number = 3.387
Exhaust density = 0.137 kg/m3
Throat-Exit area ratio = 20.665
Mass flow rate propellant = 8.376 kg/s

Specific impulse = 311.143 s
Engine Thrust = 25.566 kN
Exit area = 0.020 m2
Exit diameter = 0.160 m


In [82]:
# Now re-run with final fixed parameters
chamber_temp = 2273.15
chamber_P = 14800
gamma_exhaust = 1.15
throat_area = 0.001
Mwt_exhaust = 16.3
nozzle_P = 101.3
burn_time = 150
structure_mass = 2000
payload_mass = 100
launch_angle = 80
n_engines = 9

x = (Mwt_exhaust, gamma_exhaust, chamber_temp, nozzle_P, chamber_P, throat_area)
y = solve_thrust(x)
print_thrust(x,y)


z = solve_burnout(y, n_engines, burn_time, structure_mass, payload_mass, launch_angle)
print_burnout(z)

----------------------------------------------------------------------

Exhaust molar weight = 16.300 kg/kmol
Exhaust specific heat ratio = 1.150
Exhaust gas constant = 0.510
Chamber temperature = 2273.150 K
Nozzle exit pressure = 101.300 kPa
Chamber pressure = 14.800 MPa
Throat area = 0.001 m2
Throat diameter = 0.036 m

Nozzle velocity = 2915.187 m/s
Exhaust temperature = 1186.539 K
Mach number = 3.494
Exhaust density = 0.167 kg/m3
Throat-Exit area ratio = 17.990
Mass flow rate propellant = 8.778 kg/s

Specific impulse = 297.165 s
Engine Thrust = 25.589 kN
Exit area = 0.018 m2
Exit diameter = 0.151 m

Propellant mass = 11850.186 kg
Total mass = 13950.186 kg
Mass ratio = 6.643
Burnout height = 181.859 km
Velocity increment = 6.983 km/s
